In [8]:
from vedo import embedWindow, Plotter, show  # <- this will be used to render an embedded scene 
from itkwidgets import view
from imio import load
from vedo import Volume, Points, VolumeSlice
from vedo.applications import RayCastPlotter, IsosurfaceBrowser
import os
import pandas
import torch
import random
import nibabel as nib
import numpy as np

In [2]:
def load_atlas(mask_fn):
    orig_roi_mask = torch.from_numpy(np.transpose(nib.load(mask_fn).get_fdata(), (2, 0, 1))[:, :, ::-1].copy())
    roi_mask = orig_roi_mask.clone()
    assert (roi_mask==orig_roi_mask).all()
    roi_remap = {}
    for newi, i in enumerate(roi_mask.unique()):
        roi_remap[newi] = i
        roi_mask[roi_mask == i] = newi
    roi_mask = roi_mask.long()
    return roi_mask

In [3]:
pair_tag = 'pair16'
brain_tag = 'L74D769P4'
r = '/lichtman/Felix/Lightsheet/P4/%s/output_%s/registered' % (pair_tag, brain_tag)
mask_fn = os.path.join(r, '%s_MASK_topro_25_all.nii' % brain_tag)
# mydata = load.load_any(mask_fn)
mydata = np.transpose(nib.load(mask_fn).get_fdata(), (2, 0, 1))[:, :, ::-1].copy()
# mydata = load_atlas(mask_fn)
vol = Volume(mydata)
mesh = vol.isosurface()
mesh.alpha = 0.5
center = '/lichtman/ziquanw/Lightsheet/results/P4/center_pts_%s_%s_RoI16001.csv' % (pair_tag, brain_tag)
center = torch.from_numpy(pandas.read_csv(center).to_numpy())
# cid = torch.arange(center.shape[0]).tolist()
# random.shuffle(cid)
# center = center[cid[:1000000]]
mshape = mydata.shape
center[:,0] = center[:,0] * mshape[0]
center[:,1] = center[:,1] * mshape[1]
center[:,2] = center[:,2] * mshape[2]
print(center.shape)


torch.Size([10127514, 3])


In [4]:
loc = torch.arange(mshape[0]*mshape[1]*mshape[2]).reshape(*mshape)
density = torch.zeros(mshape, dtype=torch.long)

In [5]:
loc = loc[tuple(center.long().T)]
loc_count = loc.bincount()
loc_count = loc_count[loc_count!=0]
density[tuple(center[loc.unique()].long().T)] = loc_count
# density = density.transpose(2, 0)

In [9]:
# from brainrender.scene import Scene
# from brainrender.actors import Points
embedWindow('k3d')  # <- this will make your scene embed with k3d

# # initialise brainrender scene
# scene = Scene()
# scene.jupyter = True
# # create points actor
# cells = Points(center.numpy(), radius=45, colors="palegoldenrod", alpha=0.8)

# # visualise injection site (retrosplenial cortex)
# scene.add_brain_region(["RSPd"], color="mediumseagreen", alpha=0.6)
# scene.add_brain_region(["RSPv"], color="purple", alpha=0.6)
# scene.add_brain_region(["RSPagl"], color="mediumseagreen", alpha=0.6)

# # Add cells
# scene.add(cells)

# scene.render()

#  to actually display the scene we use `vedo`'s `show` method to show the scene's actors
# plt = Plotter()
# plt.show(*scene.renderables)  # same as vedo.show(*scene.renderables)
# z = center[:, 0]
# y = center[:, 1]
# x = center[:, 2]
# cells = Points(torch.stack([x, y, z]).numpy(), r=9)#.addPointArray(scals, name='scals')
# densecloud = cells.densify(0.1, closest=10, niter=1) # return a new pointcloud.Points
# print(cells.N(), densecloud.N())
# plt.show(mesh, cells)
# vol = Volume(density).printHistogram(bins=30, logscale=True)
# lego = vol.legosurface(vmin=1, cmap='jet')
# lego.addScalarBar(horizontal=True, c='k')
# plt.show(mesh, lego)
vol = Volume(density)
# vsl = VolumeSlice(vol)
# vsl.colorize().lighting(window=100, level=25)
plt = IsosurfaceBrowser(vol) 
plt.show(axes=7, bg2='lb')
# plt.show(mesh, vsl)

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

## Embeddded scene

In [ ]:
# embedWindow('k3d')  # <- this will make your scene embed with k3d

# # Create a scene
# scene = Scene()  # note that the title will not actually display
# shape = scene.atlas.space.shape
# print(scene.atlas.space)
# center[:, 0] = 1-center[:, 0]
# center = center + 0.1
# center[:, 0] = center[:, 0]*shape[0]*20
# center[:, 1] = center[:, 1]*shape[1]*20
# center[:, 2] = center[:, 2]*shape[2]*20
# # center = Points(center.numpy())
# actor = actors.Points(center.numpy())

# # scene.add_brain_region('MOs')

# # Make sure it gets embedded in the window
# scene.jupyter = True

# scene.add(actor)
# # scene.render now will prepare the scene for rendering, but it won't render anything yet
# scene.render()

# #  to actually display the scene we use `vedo`'s `show` method to show the scene's actors
# plt = Plotter()
# plt.show(*scene.renderables)  # same as vedo.show(*scene.renderables)